<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/FPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

replace GTEA61/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
#!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [ ]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

In [ ]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [ ]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
NUM_CLASSES = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
out_dir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
MEMSIZE = 512

In [ ]:
%reload_ext tensorboard
!rm -rf ./runs/exp1/stage


In [ ]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.objectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
import argparse
import sys

In [ ]:
# Dir for saving models and log files
model_folder = os.path.join('./', "out_dir", "./GTEA61", 'rgb')

if not os.path.isdir('./out_dir'):
  if os.path.exists(model_folder):
    print('Directory {} exists!'.format(model_folder))
    sys.exit()
    
  os.makedirs(model_folder)

writer = SummaryWriter('runs/exp1/stage/1')
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

In [ ]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()

In [ ]:
# Data loader

vid_seq_train = makeMmaps(train_data_dir, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeMmaps(val_data_dir,
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()



In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random


def gen_split(root_dir, stackSize):
    Dataset = []
    Labels = []
    NumFrames = []
    root_dir = os.path.join(root_dir, 'processed_frames2')
    # print for debugging
    #print(f"root_dir: {root_dir}")
    for dir_user in sorted(os.listdir(root_dir)):
      if not dir_user.startswith('.') and dir_user:
        class_id = 0
        directory = os.path.join(root_dir, dir_user)
        # print for debugging
        #print(f"directory: {directory}")
        for target in sorted(os.listdir(directory)):
          if not target.startswith('.'):
            directory1 = os.path.join(directory, target)
            # print for debugging
            #print(f"directory1: {directory1}")
            insts = sorted(os.listdir(directory1))
            # print for debugging
            #print(f"insts: {insts}")
            if insts != []:
               for inst in insts:
                 if not inst.startswith('.'):
                   # adding "rgb" to path becasue after the number there are
                   # both "rgb" and "mmap" directories
                   inst = inst + "/mmaps"
                   inst_dir = os.path.join(directory1, inst)
                   # print for debugging
                   print(f"inst_dir: {inst_dir}")
                   numFrames = len(glob.glob1(inst_dir, '*.png'))
                   # print for debugging
                   #print(f"numFrames: {numFrames}")
                   if numFrames >= stackSize:
                     Dataset.append(inst_dir)
                     Labels.append(class_id)
                     NumFrames.append(numFrames)
            class_id += 1
    return Dataset, Labels, NumFrames

class makeMmaps(Dataset):
    def __init__(self, root_dir, spatial_transform=None, seqLen=20,
                 train=True, mulSeg=False, numSeg=1, fmt='.png'):
        #Two folders of motion map have not at least five items (stackSize)
        self.images, self.labels, self.numFrames = gen_split(root_dir, 5)
        self.spatial_transform = spatial_transform
        self.train = train
        self.mulSeg = mulSeg
        self.numSeg = numSeg
        self.seqLen = seqLen
        # print for debugging
        #print(self.seqLen)
        self.fmt = fmt

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        vid_name = self.images[idx]
        label = self.labels[idx]
        numFrame = self.numFrames[idx]
        inpSeq = []
         # For debugging
        #print(numFrame, self.seqLen)
        self.spatial_transform.randomize_parameters()
        for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
          # Corrected with "rgb" instead of "image_" and zfill(4) instead of zfill(5)
          fl_name = vid_name + '/' + 'map' + str(int(np.floor(i))).zfill(4) + self.fmt
          img = Image.open(fl_name)
          # For debugging
          #print(img, fl_name)
          inpSeq.append(self.spatial_transform(img.convert('RGB')))
        inpSeq = torch.stack(inpSeq, 0)
        return inpSeq, label

vid_seq_train = makeMmaps(train_data_dir, seqLen=SEQLEN, fmt='.png')


inst_dir: ./GTEA61/processed_frames2/S1/close_chocolate/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_coffee/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_coffee/2/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_honey/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_honey/2/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_jam/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_ketchup/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_mayonnaise/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_mustard/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_mustard/2/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_peanut/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_peanut/2/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_sugar/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_sugar/2/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_water/1/mmaps
inst_dir: ./GTEA61/processed_frames2/S1/close_water/2/mmaps
inst_dir: ./GTEA61/proc

In [ ]:
class Binary(object):

    def __init__(self,threshold):
        self.threshold=threshold
    
    def __call__(self, img_tensor, inv, flow):
        img_tensor.map_(img_tensor,lambda x,_ : 1 if x>self.threshold else 0)
        return img_tensor
    
    def randomize_parameters(self):
        pass

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random



def gen_split(root_dir, stackSize, phase):
    RGB = []
    Labels = []
    Maps = []
    NumFrames = []
    root_dir = os.path.join(root_dir, 'processed_frames2') #GTEA61/processed_frames2/
    
    for dir_user in sorted(os.listdir(root_dir)): #S1/
        if dir_user=='.DS_Store': continue
        if (phase=='train') ^ (dir_user=="S2"):
            dir = os.path.join(root_dir, dir_user) #GTEA61/processed_frames2/S1/
            class_id=0
            
            for target in sorted(os.listdir(dir)): #close_choco/
                if target=='.DS_Store': continue
                dir1 = os.path.join(dir, target) #GTEA61/processed_frames2/S1/close_choco/
                #print(dir1)
                
                insts = sorted(os.listdir(dir1)) #1/
                if insts != []:
                    for inst in insts:
                        if inst=='.DS_Store': continue
                        
                        inst_dir = os.path.join(dir1, inst+"/rgb") #GTEA61/processed_frames2/S1/close_choco/1/rgb/
                        #print(inst_dir)
                        numFrames = len(glob.glob1(inst_dir, '*.png'))
                        
                        if numFrames >= stackSize:
                            RGB.append(inst_dir)
                            Labels.append(class_id)
                            NumFrames.append(numFrames)
                        #print(Labels)
                        #print(RGB)
                        #print(numFrames)
                            
                        inst_dir = os.path.join(dir1, inst+"/mmaps") #GTEA61/processed_frames2/S1/close_choco/1/mmaps/
                        
                        if numFrames >= stackSize:
                            Maps.append(inst_dir)
                class_id += 1
    return RGB, Maps, Labels, NumFrames 

class makeDataset(Dataset):
    def __init__(self, root_dir, spatial_transform=None, seqLen=20,
                 train=True, mulSeg=False, numSeg=1, fmt='.png',phase='train', regressor=False):

        self.images, self.maps, self.labels, self.numFrames = gen_split(root_dir, 5,phase)
        normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.spatial_transform0 = spatial_transform
        self.spatial_rgb= Compose([self.spatial_transform0, ToTensor(), normalize])
        
        if not(regressor):
            self.spatial_transform_map = Compose([self.spatial_transform0, Scale(7), ToTensor(), Binary(0.4)])
        else:
            self.spatial_transform_map = Compose([self.spatial_transform0, Scale(7), ToTensor()])
               
        
        self.train = train
        self.mulSeg = mulSeg
        self.numSeg = numSeg
        self.seqLen = seqLen
        self.fmt = fmt

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        vid_name = self.images[idx]
        label = self.labels[idx]
        map_name = self.maps[idx]

        numFrame = self.numFrames[idx]
        inpSeq = []
        mapSeq = []
        self.spatial_transform0.randomize_parameters()
        for i in np.linspace(1, numFrame, self.seqLen, endpoint=False):
            fl_name = vid_name + '/' + 'rgb' + str(int(np.floor(i))).zfill(4) + self.fmt
            img = Image.open(fl_name)
            flag=1
            j=i
            while(flag):
                
                maps_name = map_name + '/' + 'map' + str(int(np.floor(j))).zfill(4) + self.fmt
                try:
                    mappa = Image.open(maps_name)
                    flag=0
                except:
                    if j<=i:
                        j= 2*i-j+1 #j=i --> j=i +1 ; j=i-1 j-i=-1 --> j=i-(-1)+1
                    else:
                        j= 2*i-j #j=i+1 j-i=1 --> j=i-1
                    continue

            inpSeq.append(self.spatial_rgb(img.convert('RGB')))
            mapSeq.append(self.spatial_transform_map(mappa.convert('L'))) #Grayscale
        inpSeq = torch.stack(inpSeq, 0)
        mapSeq = torch.stack(mapSeq, 0)
        return inpSeq, mapSeq, label


In [ ]:
model_folder = os.path.join('./', out_dir, dataset, 'MS',str(stage))  # Dir for saving models and log files


IndentationError: ignored